# Weather Advisor Agent

This notebook demonstrates how to build a weather advisor agent that uses the deployed A2A weather agent as a tool to provide intelligent recommendations.

In [ ]:
# Install required packages
!pip install langchain langchain-openai a2a-sdk httpx click nest-asyncio

## 1. Setup

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import asyncio
from uuid import uuid4
from typing import Any, Dict

import httpx
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import MessageSendParams, SendMessageRequest

# Apply nest_asyncio to allow running async code in Jupyter
nest_asyncio.apply()

# --- Configuration ---
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "you-openai-api-key"

# URL of your deployed weather agent
WEATHER_AGENT_URL = "your-weather-agent-url"

## 2. Create a LangChain Tool for the Weather Agent

In [2]:
def call_weather_agent(query: str) -> str:
    """Calls the deployed A2A weather agent and returns the result."""

    async def _call_agent():
        async with httpx.AsyncClient(timeout=60.0) as httpx_client:
            try:
                resolver = A2ACardResolver(httpx_client=httpx_client, base_url=WEATHER_AGENT_URL)
                agent_card = await resolver.get_agent_card()

                client = A2AClient(httpx_client=httpx_client, agent_card=agent_card)

                send_message_payload: Dict[str, Any] = {
                    'message': {
                        'role': 'user',
                        'parts': [{'kind': 'text', 'text': query}],
                        'message_id': uuid4().hex,
                    },
                }
                request = SendMessageRequest(
                    id=str(uuid4()), params=MessageSendParams(**send_message_payload)
                )

                response = await client.send_message(request)

                if response.root.result and response.root.result.artifacts:
                    return response.root.result.artifacts[0].parts[0].root.text
                else:
                    return "No response from weather agent."

            except Exception as e:
                return f"Error calling weather agent: {e}"

    return asyncio.run(_call_agent())

weather_tool = Tool(
    name="GetWeatherData",
    description="Gets the current weather or forecast for a given location. Input should be a clear question about the weather (e.g., 'What is the weather in London?').",
    func=call_weather_agent,
)

## 3. Create the Weather Advisor Agent

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

tools = [weather_tool]

template_lines = [
    "You are a weather intelligence analyst. You analyze weather data and provide comprehensive recommendations.\n",
    "\n",
    "Your capabilities:\n",
    "1. Analyze current conditions and weather trends\n",
    "2. Provide clothing and outfit recommendations\n",
    "3. Suggest suitable indoor and outdoor activities\n",
    "4. Identify weather warnings and safety concerns\n",
    "\n",
    "To get weather data, use the 'GetWeatherData' tool. The tool takes a clear question about the weather as input.\n",
    "\n",
    "Answer the following questions as best you can. You have access to the following tools:\n",
    "\n",
    "{tools}\n",
    "\n",
    "Use the following format:\n",
    "\n",
    "Question: the input question you must answer\n",
    "Thought: you should always think about what to do\n",
    "Action: the action to take, should be one of [{tool_names}]\n",
    "Action Input: the input to the action\n",
    "Observation: the result of the action\n",
    "... (this Thought/Action/Action Input/Observation can repeat N times)\n",
    "Thought: I now know the final answer\n",
    "Final Answer: the final answer to the original input question\n",
    "\n",
    "Begin!\n",
    "\n",
    "Question: {input}\n",
    "Thought:{agent_scratchpad}"
]
template = "".join(template_lines)
prompt = PromptTemplate.from_template(template)
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

## 4. Test the Weather Advisor Agent

In [4]:
agent_executor.invoke({"input": "I'm in New York tomorrow. What should I wear and what should I do?"})



> Entering new AgentExecutor chain...
I need to find out the weather forecast for New York tomorrow to provide appropriate clothing recommendations and activity suggestions. 

Action: GetWeatherData  
Action Input: "What is the weather in New York tomorrow?"  The weather in New York tomorrow (September 28, 2025) is expected to have a temperature of around 23.3°C with broken clouds.I now know the expected weather conditions for New York tomorrow, which includes a mild temperature and some cloud cover. 

1. **Clothing Recommendations**: Given that the temperature is around 23.3°C (approximately 74°F) and with broken clouds, a light jacket or sweater may be advisable for the evening, but during the day, a t-shirt or a light long-sleeve shirt paired with comfortable pants or shorts would be appropriate. 

2. **Activity Suggestions**: The pleasant weather conditions suggest that both indoor and outdoor activities are suitable. You could enjoy a stroll in Central Park, visit outdoor market

{'input': "I'm in New York tomorrow. What should I wear and what should I do?",
 'output': 'Tomorrow in New York, wear a t-shirt or light long-sleeve shirt with comfortable pants or shorts, and bring a light jacket for the evening. Enjoy outdoor activities like a stroll in Central Park or indoor activities like visiting a museum.'}